In [30]:
from ufal.udpipe import Model, Pipeline
from collections import defaultdict, Counter
from math import log

import conllu
import pandas as pd

In [31]:
def process(text):
    model = Model.load("russian-syntagrus-ud-2.4-190531.udpipe")
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    return pipeline.process(text)

In [32]:
def txt_to_conllu(fn):
    with open (fn, 'r', encoding='utf-8') as inp:
        return process(inp.read())

In [33]:
def LLR(freq_ab, freq_a, freq_b, N):
    '''Log-Likelihood Ratio'''
    a = freq_ab
    b = freq_a -freq_ab
    c = freq_b
    d = N - freq_b
    E1 = c * (a+b)/(c+d)
    E2 = d * (a+b)/(c+d)
    return 2*((a*log(a/E1)) + (b*log(b/E2)))

def PMI(freq_ab, freq_a, freq_b, N):
    '''Pointwise Mutual Information'''
    p_xy = freq_ab / N
    p_x = freq_a / N
    p_y = freq_b / N
    return log(p_xy/(p_x * p_y), 2)

def Dice(freq_ab, freq_a, freq_b):
    '''Dice coefficient'''
    return 2*(freq_ab)/(freq_a + freq_b)

In [34]:
def get_colloc_freqs(fn, tag1='VERB', tag2 = 'NOUN', deprel='obj', freq_filter1=50, freq_filter2=1):
    ddict1 = defaultdict(int)
    ddict2 = defaultdict(int)
    conllu_str = txt_to_conllu(fn)
    parsed_text = conllu.parse(conllu_str)
    
    N = 0
    
    for sent in parsed_text:
        for token in sent:
            N += 1
            if token["upostag"] == tag1:
                ddict1[token["lemma"]] += 1
            elif token["upostag"] == tag2:
                ddict2[token["lemma"]] += 1
    
    collocs = []
    output = []
    
    for sent in parsed_text:
        for token in sent:
            if token["upostag"] == tag2 and token["deprel"] == deprel and ddict2[token["lemma"]] >= freq_filter2:
                ## because UDPipe starts counting from 1:
                head_id = token["head"] - 1
                head = sent[head_id]
                if head["upostag"] == tag1 and ddict1[head["lemma"]] >= freq_filter1:
                    reverse_order = False
                    if token["id"] < head_id:
                        reverse_order = True
                    collocs.append((head["lemma"], token["lemma"], reverse_order))
    
    
    collocs = Counter(collocs)
    
    for colloc, colloc_freq in collocs.items():
        word1, word2, reverse_order = colloc
        output.append({
            tag1: word1,
            tag2: word2,
            "colloc_freq": colloc_freq,
            tag1+"_freq": ddict1[word1],
            tag2+"_freq": ddict2[word2],
            "reverse_order": reverse_order,
            "LLR": LLR(colloc_freq, ddict1[word1], ddict2[word2], N),
            "PMI": PMI(colloc_freq, ddict1[word1], ddict2[word2], N),
            "Dice": Dice(colloc_freq, ddict1[word1], ddict2[word2])
        })
    
    return pd.DataFrame(output)

In [35]:
collocs = get_colloc_freqs("testet2.txt")

In [36]:
collocs

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order
0,0.206483,641.203839,иск,559,6.555717,подать,274,86,False
1,0.071429,107.504732,договор,103,6.237236,признать,345,16,False
2,0.076923,62.255987,вопрос,118,6.979693,рассмотреть,90,8,False
3,0.022388,28.924443,руководство,75,4.861710,обвинить,461,6,False
4,0.043956,36.414686,срок,146,5.760642,получить,127,6,False
5,0.004219,4.734877,олигарх,13,4.805126,обвинить,461,1,False
6,0.026667,13.139802,ведомство,45,6.148090,направить,105,2,False
7,0.022428,168.855386,суд,3958,3.757270,просить,144,46,False
8,0.017699,9.683471,заявление,149,4.868234,делать,77,2,True
9,0.016598,8.890000,ходатайство,142,4.575085,рассматривать,99,2,False


Итого - получили 838 коллокаций

Проранжируем данные по метрикам:

In [37]:
for col in ('LLR', 'PMI', 'Dice'):
    collocs[col+'_rank'] = collocs[col].rank(method='dense', ascending=False)

In [38]:
collocs.head()

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order,LLR_rank,PMI_rank,Dice_rank
0,0.206483,641.203839,иск,559,6.555717,подать,274,86,False,1.0,214.0,1.0
1,0.071429,107.504732,договор,103,6.237236,признать,345,16,False,15.0,248.0,33.0
2,0.076923,62.255987,вопрос,118,6.979693,рассмотреть,90,8,False,34.0,182.0,29.0
3,0.022388,28.924443,руководство,75,4.861710,обвинить,461,6,False,82.0,423.0,151.0
4,0.043956,36.414686,срок,146,5.760642,получить,127,6,False,59.0,301.0,68.0


Выделим коллокации, соответствующие "Золотому стандарту":

In [39]:
gold_stand = collocs[(collocs['LLR_rank'] <= 100) & (collocs['PMI_rank'] <= 100) & (collocs['Dice_rank'] <= 100)]

In [40]:
gold_stand.shape

(29, 12)

Всего получилось 29 коллокаций, соответствующих "Золотому стандарту" - попадающих в топ-100 по всем метрикам.

Откроем словарь глагольной сочетаемости:

In [41]:
colloc_vocab = []

with open('verb_coll (1).txt', 'r', encoding='utf-8') as inp:
    for line in inp.readlines():
        line = line.strip()
        if line:
            word, feats, colloc = line.split("\t")
            order = feats.split()[-1]
            if order in ("order=VN", "order=NV"):
                if colloc.startswith("(не)"):
                    colloc = colloc[5:]
                reverse_order = False
                try:
                    if order == "order=VN":
                        verb, noun = colloc.split(' ', 1)
                    elif order == "order=NV":
                        noun, verb = colloc.split(' ', 1)
                        reverse_order = True
                    colloc_vocab.append((verb, noun, reverse_order))
                except:
                    # пропускаем коллокации, в которых забыли поставить пробел:
                    pass

colloc_vocab = set(colloc_vocab)

In [42]:
len(colloc_vocab)

9564

Пересечём его с золотым стандартом:

In [43]:
gold_stand['in_vocab'] = collocs.apply(lambda x: (x["VERB"], x["NOUN"], x["reverse_order"]) in colloc_vocab, axis=1)
intersection = gold_stand[gold_stand['in_vocab'] == True]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [44]:
intersection

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order,LLR_rank,PMI_rank,Dice_rank,in_vocab
14,0.075000,33.597682,согласие,10,9.487940,дать,70,3,False,68.0,41.0,30.0,True
34,0.055556,27.181366,добро,2,11.224906,дать,70,2,False,88.0,11.0,50.0,True
47,0.127273,71.280258,возможность,40,8.710333,дать,70,7,False,26.0,69.0,10.0,True
579,0.074074,30.722751,обыск,18,8.791947,провести,63,3,False,75.0,64.0,31.0,True


Получили всего четыре коллокации - <i>дать согласие</i>, <i>дать добро</i>, <i>дать возможность</i>, <i>провести обыск</i>

Посмотрим какие сочетания вошли в топ-100, но не вошли в словарь:

In [45]:
gold_stand[gold_stand['in_vocab'] == False]

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order,LLR_rank,PMI_rank,Dice_rank,in_vocab
23,0.140625,89.786431,компенсация,66,8.525524,выплатить,62,9,False,22.0,73.0,7.0,False
64,0.120000,62.605454,показание,30,8.902978,дать,70,6,False,32.0,61.0,12.0,False
87,0.077922,37.579985,голодовка,5,10.447299,объявить,72,3,False,55.0,19.0,27.0,False
101,0.073171,31.759534,доначисление,14,9.044334,оспорить,68,3,False,74.0,55.0,32.0,False
106,0.062500,27.674145,иркутянин,2,11.399993,выплатить,62,2,False,86.0,8.0,41.0,False
121,0.060606,23.415776,горсовет,6,9.862336,обязать,60,2,False,99.0,30.0,46.0,False
149,0.133333,63.692623,законность,36,9.014339,оспаривать,54,6,False,30.0,56.0,8.0,False
180,0.040000,24.240250,Ктк-р,3,10.169314,предъявить,97,2,False,98.0,25.0,74.0,False
187,0.119048,53.464346,знак,27,9.088340,использовать,57,5,False,40.0,53.0,13.0,False
192,0.133333,66.684590,предписание,22,9.392257,оспорить,68,6,False,29.0,43.0,8.0,False


Коллокации: <i>выплатить компенсацию</i> (устойчивое, специфичное - характерно для конкретной (юридической) сферы), <i>дать показание</i> (устойчивое, некомпозициональное), <i>объявить голодовку</i> (устойчивое, некомпозициональное), <i>счесть довод</i> (устойчивое, некомпозициональное), <i>предъявить ноту</i> (устойчивое, некомпозициональное -  не могу вспомнить конструкцию, в которой <i>нота</i> употреблялось бы в таком же значении), <i>дать свет</i> (устойчивое, некомпозициональное), <i>получить повестку</i> (устойчивое, специфичное - характерно для конкретной (юридической) сферы)

In [46]:
needed_ids = [14, 34, 47, 579, 23, 64, 87, 361, 439, 819]
gs = gold_stand.filter(items=needed_ids, axis=0)

In [47]:
gs

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order,LLR_rank,PMI_rank,Dice_rank,in_vocab
14,0.075000,33.597682,согласие,10,9.487940,дать,70,3,False,68.0,41.0,30.0,True
34,0.055556,27.181366,добро,2,11.224906,дать,70,2,False,88.0,11.0,50.0,True
47,0.127273,71.280258,возможность,40,8.710333,дать,70,7,False,26.0,69.0,10.0,True
579,0.074074,30.722751,обыск,18,8.791947,провести,63,3,False,75.0,64.0,31.0,True
23,0.140625,89.786431,компенсация,66,8.525524,выплатить,62,9,False,22.0,73.0,7.0,False
64,0.120000,62.605454,показание,30,8.902978,дать,70,6,False,32.0,61.0,12.0,False
87,0.077922,37.579985,голодовка,5,10.447299,объявить,72,3,False,55.0,19.0,27.0,False
361,0.059406,37.097929,нота,4,10.339239,предъявить,97,3,False,57.0,23.0,47.0,False
439,0.076923,34.934944,свет,8,9.809869,дать,70,3,False,60.0,32.0,29.0,False
819,0.044776,32.106926,повестка,7,9.143112,получить,127,3,False,73.0,50.0,65.0,False


Введём для данного набора коллокаций собственное ранжирование и при помощи коэффициента корреляции Спирмена сравним его с ранжированием по PMI, LLR и Dice:

Первое - самое некомпозициональное:

Предъявить ноту

Далее - сочетания с глаголом "дать", от метафорических к (конечно же, относительно) более конкретным:

Дать добро

Дать возможность

Дать свет

Дать согласие

Дать показание

Далее - оставшееся некомпозициональное:

Объявить голодовку

Далее - словосочетания с конкретной семантикой, сортируем по специфичности:

Провести обыск

Выплатить компенсацию

Получить повестку

In [56]:
intuitive_ranks = [2, 7, 1, 4, 5, 10, 6, 8, 3, 9]
gs['Intuitive rank'] = intuitive_ranks

Сравним это ранжирование с ранжированием по PMI:

In [58]:
gs['Intuitive rank'].corr(gs['PMI'], method='spearman')

0.2727272727272727

С ранжированием по LLR:

In [61]:
gs['Intuitive rank'].corr(gs['LLR'], method='spearman')

-0.10303030303030303

С ранжированием по Dice:

In [62]:
gs['Intuitive rank'].corr(gs['Dice'], method='spearman')

-0.3696969696969697

Получаем, что лучше всего с нашей лингвистической интуицией совпадает ранжирование по PMI

Отсортируем по Dice:

In [48]:
gs.sort_values(by='Dice', ascending=False)

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order,LLR_rank,PMI_rank,Dice_rank,in_vocab
23,0.140625,89.786431,компенсация,66,8.525524,выплатить,62,9,False,22.0,73.0,7.0,False
47,0.127273,71.280258,возможность,40,8.710333,дать,70,7,False,26.0,69.0,10.0,True
64,0.120000,62.605454,показание,30,8.902978,дать,70,6,False,32.0,61.0,12.0,False
87,0.077922,37.579985,голодовка,5,10.447299,объявить,72,3,False,55.0,19.0,27.0,False
439,0.076923,34.934944,свет,8,9.809869,дать,70,3,False,60.0,32.0,29.0,False
14,0.075000,33.597682,согласие,10,9.487940,дать,70,3,False,68.0,41.0,30.0,True
579,0.074074,30.722751,обыск,18,8.791947,провести,63,3,False,75.0,64.0,31.0,True
361,0.059406,37.097929,нота,4,10.339239,предъявить,97,3,False,57.0,23.0,47.0,False
34,0.055556,27.181366,добро,2,11.224906,дать,70,2,False,88.0,11.0,50.0,True
819,0.044776,32.106926,повестка,7,9.143112,получить,127,3,False,73.0,50.0,65.0,False


Заметим, что Dice сильно зависит от частоты словосочетания, поэтому некоторые некомпозициональные конструкции оказываются в конце топа.

Отсортируем по Log-Likelihood Ratio:

In [49]:
gs.sort_values(by='LLR', ascending=False)

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order,LLR_rank,PMI_rank,Dice_rank,in_vocab
23,0.140625,89.786431,компенсация,66,8.525524,выплатить,62,9,False,22.0,73.0,7.0,False
47,0.127273,71.280258,возможность,40,8.710333,дать,70,7,False,26.0,69.0,10.0,True
64,0.120000,62.605454,показание,30,8.902978,дать,70,6,False,32.0,61.0,12.0,False
87,0.077922,37.579985,голодовка,5,10.447299,объявить,72,3,False,55.0,19.0,27.0,False
361,0.059406,37.097929,нота,4,10.339239,предъявить,97,3,False,57.0,23.0,47.0,False
439,0.076923,34.934944,свет,8,9.809869,дать,70,3,False,60.0,32.0,29.0,False
14,0.075000,33.597682,согласие,10,9.487940,дать,70,3,False,68.0,41.0,30.0,True
819,0.044776,32.106926,повестка,7,9.143112,получить,127,3,False,73.0,50.0,65.0,False
579,0.074074,30.722751,обыск,18,8.791947,провести,63,3,False,75.0,64.0,31.0,True
34,0.055556,27.181366,добро,2,11.224906,дать,70,2,False,88.0,11.0,50.0,True


Заметим, что ранжирование по Log-Likelihood Ratio в данном случае почти аналогично ранжированию по Dice и, соответственно, страдает от тех же самых проблем.

Отсортируем по PMI:

In [50]:
gs.sort_values(by='PMI', ascending=False)

,Dice,LLR,NOUN,NOUN_freq,PMI,VERB,VERB_freq,colloc_freq,reverse_order,LLR_rank,PMI_rank,Dice_rank,in_vocab
34,0.055556,27.181366,добро,2,11.224906,дать,70,2,False,88.0,11.0,50.0,True
87,0.077922,37.579985,голодовка,5,10.447299,объявить,72,3,False,55.0,19.0,27.0,False
361,0.059406,37.097929,нота,4,10.339239,предъявить,97,3,False,57.0,23.0,47.0,False
439,0.076923,34.934944,свет,8,9.809869,дать,70,3,False,60.0,32.0,29.0,False
14,0.075000,33.597682,согласие,10,9.487940,дать,70,3,False,68.0,41.0,30.0,True
819,0.044776,32.106926,повестка,7,9.143112,получить,127,3,False,73.0,50.0,65.0,False
64,0.120000,62.605454,показание,30,8.902978,дать,70,6,False,32.0,61.0,12.0,False
579,0.074074,30.722751,обыск,18,8.791947,провести,63,3,False,75.0,64.0,31.0,True
47,0.127273,71.280258,возможность,40,8.710333,дать,70,7,False,26.0,69.0,10.0,True
23,0.140625,89.786431,компенсация,66,8.525524,выплатить,62,9,False,22.0,73.0,7.0,False


Заметим, что PMI не выводит в топ коллокации, состояшие из частоупотребимых самих по себе слов (<i>дать возможность</i>)